In [1]:
# imports
from collections import Counter
from itertools import islice
import re
import pandas as pd
import string
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
# #########################
# DATA CLEANING TECHNIQUES
# #########################
# split by sentance
# remove puncuation
# lowercase letters
# remove/convert numbers
# -------------------------

# #########################
# NLP TECHNIQUES
# #########################
# stemming
# -------------------------

In [2]:
stopwords = {"I", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "can", "will", "just", "don", "should", "now"}
# sent1 = "i am sam"
# sent2 = "sam i am"
# sent3 = "i do not like green eggs and ham"
# sent4 = "the quick person did not realize his speed and the quick person bumped "


# # words = re.findall("\w+", sent4)
# # bigramCount = dict(Counter(zip(words, islice(words, 1, None))))

# # for i in bigramCount.keys():
# #     print(str(i) + " count: %d" % bigramCount[i])

# load in the crawled data and perform some cleaning
news = pd.read_csv(r"C:\Users\Rogith\Desktop\Notebooks\news2.csv")
news = news[:100]

In [3]:
gibberish = ['Getty Images', '\\\\xc3', '\\\\x94', '\\\\x9c', '\\\\x9d', '\\\\x95', '\\\\xb3', '\\\\xe2', '\\\\x80', '\\\\x99', '\\\\xa9', '\\\\xf0', '\\\\x9f', '\\\\x91']
for i in range(len(gibberish)):
    for j in range(news.shape[0]):
        output = news.iloc[j][4]
        output = re.sub(gibberish[i], '', output)
        output = re.sub('\\\\n', ' ', output)
        output = re.sub('\\\\\\\'', '\'', output)
        news.iloc[j][4] = output

print("#################################################")

#################################################


In [4]:
collection = str.maketrans('', '', string.punctuation)
dterms = dict()
for j in range(news.shape[0]):    
    tokens = word_tokenize(news.iloc[j][4])
    tokens = [t.lower() for t in tokens]
    depunc = [t.translate(collection) for t in tokens]
    words = [w for w in depunc if not w  == '']
    cleaner = [w for w in words if not w in stopwords]
    stemmed = [porter.stem(word) for word in cleaner]
    dterms[j] = stemmed


In [5]:
# initialize the set and the dataframe with the first word of the first document
words_encountered = set(dterms[0][0])
unigrams = pd.DataFrame({dterms[0][0]: [0]})

for i in range(news.shape[0]):
# for i in range(4):
    #create new row in unigrams
    if i > 0:
        tdf = pd.DataFrame([[0]*len(unigrams.columns)], columns=unigrams.columns)
        t = unigrams.append(tdf,ignore_index=True)
        unigrams = t
#     j = 0
    for word in dterms[i]:
        if word not in words_encountered:
            words_encountered.add(word)
            col_index = len(unigrams.columns)
            temp = pd.DataFrame({word: [0]*(i+1)})
            t2 = unigrams.join(temp)
            unigrams = t2
            unigrams.at[i, word] = 1
        else:
            cols = list(unigrams.columns)
            unigrams.at[i, word] += 1
# print(unigrams)
#         j += 1
#         if j == 3:
#             break
print("print when finished")

print when finished


In [6]:
unigrams.to_csv('unigrams.csv', encoding='utf-8')